# ndarray

> The NDArray class in minima is a kind of container that can hold all sorts of elements, especially numbers (but for now just f32 numbers).  
> It can work with data that has any number of dimensions. For example, it can handle simple 1-dimensional rows or columns, more complex 2-dimensional  
>  matrices, or even arrays with more dimensions. The cool thing about this NDArray class is that it can work with different backends.  
> So, you could use it with Numpy, or a CPU backend, or even a GPU backend. It's a versatile tool for handling multi-dimensional data.  

In [ ]:
#| default_exp ndarray

In [ ]:
#| export
import math
import numpy as np
from minima import ndarray_backend_numpy
from typing import Optional, Sequence, Tuple, Union, Callable, Any
from minima.utility import *
# from . import ndarray_backend_cpu

In [ ]:
#| export
class BackendDevice:
    """A backend device, wraps the implementation module."""

    def __init__(self, name, mod):
        self.name = name
        self.mod = mod

    def __eq__(self, other):
        return self.name == other.name

    def __repr__(self):
        return f"device(type='{self.name}')"

    def __getattr__(self, name):
        return getattr(self.mod, name)

    def enabled(self):
        return self.mod is not None

    def randn(self, *shape, dtype="float32"):
        return NDArray(numpy.random.randn(*shape).astype(dtype), device=self)

    def rand(self, *shape, dtype="float32"):
        return NDArray(numpy.random.rand(*shape).astype(dtype), device=self)

    def one_hot(self, n, i, dtype="float32"):
        return NDArray(numpy.eye(n, dtype=dtype)[i], device=self)

    def empty(self, shape, dtype="float32"):
        dtype = "float32" if dtype is None else dtype
        assert dtype == "float32"
        return NDArray.make(shape, device=self)

    def full(self, shape, fill_value, dtype="float32"):
        dtype = "float32" if dtype is None else dtype
        assert dtype == "float32"
        arr = self.empty(shape, dtype)
        arr.fill(fill_value)
        return arr

def cpu_numpy():
    """Return numpy device"""
    return BackendDevice('cpu_numpy', ndarray_backend_numpy)

def default_device():
    return cpu_numpy()

In [ ]:
#| export
class NDArray:

    """
    NDArray represents a n-dimensional array with operations that can be performed on multiple devices. 
    This class is an abstraction over numpy and other backend devices, providing a unified interface to interact with arrays.

    Use cases of this class include numerical operations, scientific computing, and machine learning.

    Parameters
    ----------
    value : NDArray or np.ndarray or other array-like structures
        The array-like structure to be transformed into NDArray.
    device : Optional[BackendDevice]
        The device on which the array computations should be performed. 
        If None, the default device is used.

    Attributes
    ----------
    _shape : tuple
        The shape of the array.
    _strides : tuple
        The strides of the array.
    _offset : int
        The offset in the underlying buffer.
    _device : BackendDevice
        The device on which the array computations are performed.
    _handle : Buffer
        The underlying buffer that holds the data.
    """
    
    def __init__(
        self,
        value: Union['NDArray', np.ndarray, Sequence], # The value on which to create the NDArray from
        device: Optional[BackendDevice] = None # The device on which the array computations are performed.
    ) -> None:
        """
        Constructs a new NDArray instance from an existing `NDArray`, numpy array, or a Python sequence. 
        This array can be used to perform high-performance computations on the specified device.

        Parameters
        ----------
        value : Union[NDArray, np.ndarray, Sequence]
            The value to create the NDArray from. If it's an NDArray, it is deep-copied to the new NDArray. 
            If it's a numpy array, it's copied to a new NDArray. If it's a Python sequence, it's converted to 
            a numpy array and then copied to a new NDArray.

        device : Optional[BackendDevice]
            The device on which the array computations are performed. Defaults to the device of the input value 
            if it's an NDArray, or to the default device otherwise.
        """
        
        if isinstance(value, NDArray): # copy of existing NDArray
            if device is None: device = value.device
            self._init(value.to(device) + 0.0)
        elif isinstance(value, np.ndarray): # copy of existing np array
            device = device if device is not None else default_device()
            array = self.make(value.shape, device=device)
            array.device.from_numpy(np.ascontiguousarray(value), array._handle)
            self._init(array)
        else:
            array = NDArray(np.array(value), device=device)
            self._init(array)

    def _init(self, other) -> None:
        """
        A private method that initializes the new NDArray with the values, shape, strides, offset, device, 
        and handle of another NDArray.
    
        Parameters
        ----------
        other : NDArray
            The NDArray to initialize from.
        """
        self._shape = other._shape
        self._strides = other._strides
        self._offset = other._offset
        self._device = other._device
        self._handle = other._handle

    @staticmethod
    def make(
        shape: Sequence[int], # The shape of the new array.
        strides: Optional[Sequence[int]] = None, # The strides of the new array. If None, compact strides are computed.
        device: Optional[BackendDevice] = None, # The device on which the new array computations should be performed. If None, the default device is used.
        offset: Optional[int] = None, # The offset in the underlying buffer of the new array. If None, it defaults to 0.
        handle: Optional[Any] = None # The underlying buffer that should hold the data. If None, a new buffer is allocated.
    ) -> 'NDArray':
        """
        Constructs a new NDArray with the specified shape, strides, device, offset, and handle.

        Parameters
        ----------
        shape : Sequence[int]
            The shape of the new array.
        strides : Optional[Sequence[int]]
            The strides of the new array. If None, compact strides are computed.
        device : Optional[BackendDevice]
            The device on which the new array computations should be performed. If None, the default device is used.
        offset : Optional[int]
            The offset in the underlying buffer of the new array. If None, it defaults to 0.
        handle : Optional[Buffer]
            The underlying buffer that should hold the data. If None, a new buffer is allocated.

        Returns
        -------
        NDArray
            A new NDArray instance.
        """
        array = NDArray.__new__(NDArray)
        array._shape = tuple(shape)
        array._strides = NDArray.compact_strides(shape) if strides is None else strides
        array._device = default_device() if device is None else device
        array._offset = offset
        array._handle = array.device.Array(prod(shape)) if handle is None else handle
        return array

    @staticmethod
    def compact_strides(shape) -> Tuple:
        res = [1] + [prod(shape[-i:]) for i in range(1, len(shape))]
        return tuple(res[::-1])

    def _is_compact(self) -> bool:
        return self.strides == self.compact_strides(self.shape) and prod(self.shape) == self._handle.size

    def compact(self) -> 'NDArray':
        """
        Returns a compact version of this array. If the array is already compact, it returns itself.

        Returns
        -------
        NDArray
            The compact version of this array.
        """
        if self._is_compact():
            return self
        out = NDArray.make(shape=self.shape, device=self.device)
        self.device.compact(self._handle, out._handle, self.shape, self.strides, self.offset)
        return out

    @property
    def shape(self) -> Tuple[int, ...]:
        return self._shape

    @property
    def strides(self) -> Tuple[int, ...]:
        return self._strides

    @property
    def device(self) -> BackendDevice:
        return self._device

    @property
    def dtype(self) -> str:
        # only support float32 for now
        return "float32"

    @property
    def ndim(self) -> int:
        """ Return number of dimensions. """
        return len(self._shape)

    @property
    def size(self) -> int:
        return prod(self._shape)

    def __repr__(self) -> str:
        return "NDArray(" + self.numpy().__str__() + f", device={self.device})"

    def __str__(self) -> str:
        return self.numpy().__str__()

    def __add__(self, other: Union['NDArray', float]) -> 'NDArray':
        """
        Performs element-wise addition between this array and `other`. If `other` is not an NDArray, it is treated as a scalar.

        Parameters
        ----------
        other : NDArray or scalar
            The other operand in the addition.

        Returns
        -------
        NDArray
            The result of the addition.

        Raises
        ------
        AssertionError
            If `other` is an NDArray and does not have the same shape as this array.
        """
        return self._ewise_or_scalar(other, ewise_fn=self.device.ewise_add, scalr_fn=self.device.scalar_add)

    def _ewise_or_scalar(self, other: Union['NDArray', float], ewise_fn: Callable, scalr_fn: Callable) -> 'NDArray':
        """
        This private method applies an element-wise function (`ewise_fn`) to two `NDArray` instances, or a scalar function (`scalr_fn`) 
        to this `NDArray` and a scalar value. It returns a new `NDArray` instance with the results.
    
        Parameters
        ----------
        other : Union[NDArray, float]
            The second operand for the operation. It can be either another `NDArray` (for element-wise operations) or a scalar 
            (for scalar operations).
    
        ewise_fn : Callable
            A function to apply element-wise if `other` is an `NDArray`. This function should take two `NDArray` handles and 
            output a handle.
    
        scalr_fn : Callable
            A function to apply if `other` is a scalar. This function should take an `NDArray` handle and a scalar, and 
            output a handle.
    
        Returns
        -------
        NDArray
            A new `NDArray` instance with the results of the operation.
    
        Raises
        ------
        AssertionError
            If `other` is an `NDArray` but does not have the same shape as `self`.
        """
        out = NDArray.make(shape=self.shape, device=self.device)
        if isinstance(other, NDArray):
            assert self.shape == other.shape, f'operands could not be added together with shapes {self.shape} {other.shape}'
            ewise_fn(self.compact()._handle, other.compact()._handle, out._handle)
        else:
            scalr_fn(self.compact()._handle, other, out._handle)
        return out
            
    def to(self, device: BackendDevice) -> 'NDArray':
        """
        Transfers this array to the specified device.

        Parameters
        ----------
        device : BackendDevice
            The device to which this array should be transferred.

        Returns
        -------
        NDArray
            This array after it has been transferred to `device`.
        """
        return self if device == self.device else NDArray(self.numpy(), device=device)
        
    def numpy(self) -> np.ndarray:
        """
        Returns a numpy representation of this array.

        Returns
        -------
        np.ndarray
            A numpy array that has the same data as this array.
        """
        return self.device.to_numpy(self._handle, self._shape, self._strides, self._offset)

In [ ]:
x = NDArray([1, 2, 3, 4, 5])
import torch; t = torch.tensor([1, 2, 3, 4, 5])

In [ ]:
x.shape

(5,)

In [ ]:
import nbdev; nbdev.nbdev_export()